In [1]:
from extractor import NYTimesSpider
import requests

In [2]:
spider = NYTimesSpider(pages_to_parse=1)

In [3]:
print('testing token extraction')
tokens = spider._get_tokens()
print(f'Tokens: {tokens}')

testing token extraction
Tokens: {'nyt-token': 'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAs+/oUCTBmD/cLdmcecrnBMHiU/pxQCn2DDyaPKUOXxi4p0uUSZQzsuq1pJ1m5z1i0YGPd1U1OeGHAChWtqoxC7bFMCXcwnE1oyui9G1uobgpm1GdhtwkR7ta7akVTcsF8zxiXx7DNXIPd2nIJFH83rmkZueKrC4JVaNzjvD+Z03piLn5bHWU6+w+rA+kyJtGgZNTXKyPh6EC6o5N+rknNMG5+CdTq35p8f99WjFawSvYgP9V64kgckbTbtdJ6YhVP58TnuYgr12urtwnIqWP9KSJ1e5vmgf3tunMqWNm6+AnsqNj8mCLdCuc5cEB74CwUeQcP2HQQmbCddBy2y0mEwIDAQAB', 'nyt-app-type': 'project-vi', 'nyt-app-version': '0.0.5'}


In [4]:
spider.header_update()

In [5]:
spider_headers = spider.headers

In [6]:
spider_headers

{'User-Agent': 'Mozilla/5.0',
 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
 'nyt-token': 'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAs+/oUCTBmD/cLdmcecrnBMHiU/pxQCn2DDyaPKUOXxi4p0uUSZQzsuq1pJ1m5z1i0YGPd1U1OeGHAChWtqoxC7bFMCXcwnE1oyui9G1uobgpm1GdhtwkR7ta7akVTcsF8zxiXx7DNXIPd2nIJFH83rmkZueKrC4JVaNzjvD+Z03piLn5bHWU6+w+rA+kyJtGgZNTXKyPh6EC6o5N+rknNMG5+CdTq35p8f99WjFawSvYgP9V64kgckbTbtdJ6YhVP58TnuYgr12urtwnIqWP9KSJ1e5vmgf3tunMqWNm6+AnsqNj8mCLdCuc5cEB74CwUeQcP2HQQmbCddBy2y0mEwIDAQAB',
 'nyt-app-type': 'project-vi',
 'nyt-app-version': '0.0.5'}

In [7]:
spider.start_urls

['https://www.nytimes.com/section/business/media']

In [8]:
print("Testing request generation...")
endpoint = spider._request_generator(spider.start_urls[0])
print(f"Endpoint: {endpoint}")

Testing request generation...
Endpoint: https://samizdat-graphql.nytimes.com/graphql/v2?operationName=CollectionsQuery&variables=%7B%22id%22%3A%22/section/business/media%22%2C%22first%22%3A10%2C%22exclusionMode%22%3A%22HIGHLIGHTS_AND_EMBEDDED%22%2C%22isFetchMore%22%3Afalse%2C%22isTranslatable%22%3Afalse%2C%22isEspanol%22%3Afalse%2C%22highlightsListUri%22%3A%22nyt%3A//per/personalized-list/__null__%22%2C%22highlightsListFirst%22%3A0%2C%22hasHighlightsList%22%3Afalse%2C%22cursor%22%3Anull%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%228334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8%22%7D%7D


In [9]:
print("Testing API connection...")
connection_ok = spider._check_api_connection(endpoint)
print(f"Connection OK: {connection_ok}")

Testing API connection...
Connection OK: True


In [10]:
r = spider.start_requests()

In [11]:
output = spider.parse(r)

In [16]:
r = spider.session.get(endpoint)

In [19]:
ex_response = r.json()

In [29]:
ex_response['data']['legacyCollection']['url']

'https://www.nytimes.com/section/business/media'

In [33]:
end_cursor = ex_response['data']['legacyCollection']['collectionsPage']['stream']['pageInfo']['endCursor']

In [34]:
next_endpoint  = spider._request_generator(url="https://nytimes.com/section/business/media",cursor=end_cursor)

In [35]:
next_endpoint

'https://samizdat-graphql.nytimes.com/graphql/v2?operationName=CollectionsQuery&variables=%7B%22id%22%3A%22/section/business/media%22%2C%22first%22%3A10%2C%22exclusionMode%22%3A%22HIGHLIGHTS_AND_EMBEDDED%22%2C%22isFetchMore%22%3Afalse%2C%22isTranslatable%22%3Afalse%2C%22isEspanol%22%3Afalse%2C%22highlightsListUri%22%3A%22nyt%3A//per/personalized-list/__null__%22%2C%22highlightsListFirst%22%3A0%2C%22hasHighlightsList%22%3Afalse%2C%22cursor%22%3A%22YXJyYXljb25uZWN0aW9uOjk%3D%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%228334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8%22%7D%7D'

In [36]:
spider.session.get(next_endpoint)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [37]:
sess = requests.Session()
sess.headers.update(spider_headers)

In [38]:
sess.headers

{'User-Agent': 'Mozilla/5.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Connection': 'keep-alive', 'nyt-token': 'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAs+/oUCTBmD/cLdmcecrnBMHiU/pxQCn2DDyaPKUOXxi4p0uUSZQzsuq1pJ1m5z1i0YGPd1U1OeGHAChWtqoxC7bFMCXcwnE1oyui9G1uobgpm1GdhtwkR7ta7akVTcsF8zxiXx7DNXIPd2nIJFH83rmkZueKrC4JVaNzjvD+Z03piLn5bHWU6+w+rA+kyJtGgZNTXKyPh6EC6o5N+rknNMG5+CdTq35p8f99WjFawSvYgP9V64kgckbTbtdJ6YhVP58TnuYgr12urtwnIqWP9KSJ1e5vmgf3tunMqWNm6+AnsqNj8mCLdCuc5cEB74CwUeQcP2HQQmbCddBy2y0mEwIDAQAB', 'nyt-app-type': 'project-vi', 'nyt-app-version': '0.0.5'}

In [39]:
r = sess.get(next_endpoint)

In [40]:
r.status_code

200

In [41]:
next_page = r.json()

In [42]:
articles = next_page['data']['legacyCollection']['collectionsPage']['stream']['edges']
for article in articles:
    article_data = article["node"]
    result = {
                "headline": article_data["headline"][
                    "default"
                ],  # Need to get text which is in default="headline"
                "summary": article_data["summary"],
                "Published Date": article_data["firstPublished"],
                "url": article_data["url"],
                "News Source": article_data["__typename"],
            }
    print(result)

{'headline': 'With $217 Million in Ticket Sales, ‘Superman’ Helps Save Warner Bros., Too', 'summary': 'The film is the latest part of a startling turnaround for the studio, which has now released five consecutive hits.', 'Published Date': '2025-07-13T15:46:49.000Z', 'url': 'https://www.nytimes.com/2025/07/13/business/media/superman-warner-bros-box-office.html', 'News Source': 'Article'}
{'headline': 'The Streaming Wars Come Down to 2: YouTube vs. Netflix', 'summary': 'The two giant video companies have far different strategies, but the same goal: controlling your TV set.', 'Published Date': '2025-07-12T09:00:46.000Z', 'url': 'https://www.nytimes.com/2025/07/12/business/media/youtube-netflix-streaming.html', 'News Source': 'Article'}
{'headline': 'David Gergen, Adviser to Presidents and Political Commentator, Dies at 83', 'summary': 'He served under Nixon, Ford, Reagan and Clinton before becoming a top editor and a familiar TV pundit. “Centrism doesn’t mean splitting the difference,” he